In [9]:
%matplotlib inline
import matplotlib.pyplot as plt

In [10]:
import scipy.misc
# scipy.misc.imsave('outfile.jpg', image_array)

In [34]:
import pandas as pd
import numpy as np
import os
import scipy
import imageio

In [12]:
import pickle

In [13]:
ls

0/                           lab-4.ipynb
32.png                       labMax.ipynb
38.png                       play_0/
ReinforcementLearning.ipynb  plays/
blkwht.png                   prediction.csv
bookdraft2018jan1.pdf        renju/
intro.ipynb                  tinkoff_math.ipynb
lab-1.ipynb                  train-1.renju
lab-3.ipynb                  train-2.renju


In [14]:
f=open("/Users/ahmad/Desktop/Renju/train-1.renju", "r")

In [15]:
data = []
i = 0
for line in f.readlines():
    if (not(i % 100000)):
        print(i)
    i += 1
    parsed = line.split(' ')
    data.append(parsed)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [16]:
def motiontonumber(motion):
    letter = ord(motion[0]) - ord('a')
    if (letter > 8):
        return (letter * 15 - 15 + int(motion[1:]))
    return (letter * 15 + int(motion[1:]))

In [ ]:
data

In [17]:
def convert(data):
    newdata = []
    for play in data:
        i = 0
        sz = len(play)
        newplay = []
        if (play[0] == "white"):
            newplay.append(0)
        elif (play[0] == "black"):
            newplay.append(1)
        else:
            continue
        for i in range(1, sz):
            newplay.append(motiontonumber(play[i]))
        newdata.append(newplay)
    return newdata

In [18]:
newdata = convert(data)

In [19]:
import pickle

In [21]:
with open("newdata.txt", "wb") as fp:   #Pickling
    pickle.dump(newdata, fp)

In [22]:
X_train = np.zeros((200000, 15, 15, 1))
Y_train = np.zeros(200000)
t = 0

In [31]:
# plt.imshow(g, cmap='Greys',  interpolation='nearest')
def createImagesFromPlay(newdata, k, folder, t):
    i = 0
    fname = folder + '/' + str(i) + '.png'
    initial = np.ones((15, 15))
    for move in newdata[k]:
        if (i == 0):
            i += 1
            continue
        fname = folder + '/' + str(i - 1) + '.png'
        if (t < 200000):
            reshaped = np.zeros((15, 15, 1))
            reshaped[:, :, 0] = initial[:, :]
            X_train[t] = reshaped
            Y_train[t] = move
            t += 1
        initial[move // 15][move % 15] = 0 + 0.5 * (i % 2)
        imageio.imsave(fname, initial)
    return t

In [32]:
def createAllImages(newdata, t):
    for i in range(12000):
        folder = str(i)
        if (not(i % 100)):
            print(str(i / 100), " PERCENT IS DONE ")
        if not os.path.exists('/Users/ahmad/Desktop/Renju/plays/' + folder):
            os.makedirs('/Users/ahmad/Desktop/Renju/plays/' + folder)
        t = createImagesFromPlay(newdata, i, ('/Users/ahmad/Desktop/Renju/plays/' + folder), t)

In [33]:
createAllImages(newdata, 0)

0.0  PERCENT IS DONE 


NameError: name 'imageio' is not defined

 If image N in folder M, then the next move is data[M][N] (because data[M][0] is a winner of the round and numeration of images starts from 1)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# Generate dummy data

model = Sequential()
model.add(Conv2D(16, (2, 2), padding='same',
                 input_shape=(15, 15, 1)))
model.add(LeakyReLU())
model.add(Conv2D(16, (2, 2)))
model.add(LeakyReLU())
model.add(BatchNormalization(axis=3))

model.add(Conv2D(32, (2, 2), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(32, (2, 2)))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=3))

model.add(Flatten())
model.add(Dense(600))
model.add(LeakyReLU())
model.add(Dropout(0.3))
model.add(Dense(225))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()
